In [1]:
import statsapi
# from google.cloud import bigquery
from datetime import date, timedelta, datetime
import requests
import time
import sys
import os
import csv
import glob
from pprint import pprint
import sqlite3
import hashlib

In [2]:
# File paths
PLAYER_STATS_FOLDER = 'stats/player/'
TEAM_STATS_FOLDER = 'stats/team/'

In [3]:


#Get today/yesterdays date 
today = date.today()
yesterday = today - timedelta(days = 1)
sunday = today - timedelta(days=today.weekday())
currentSeason = today.year
start_date = yesterday
end_date = yesterday

In [4]:
def player_stat_data(player_id, group=None, season=None, stats='season', gamePk=None):
    """
    Fetch player stats from the MLB Stats API.

    Args:
        player_id (int): The player ID.
        group (str): Stat group, e.g., 'hitting', 'pitching', or 'fielding'.
        season (int): The season year (required for 'season' stats).
        stats (str): Stat type. Either 'season' or 'gameLog'.
        gamePk (int): Optional. If provided with 'gameLog', filters to specific game splits.

    Returns:
        dict: A single stat split (for game or season), or None if not found.
    """
    url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats"
    
    params = {
        "stats": stats
    }
    
    if season:
        params["season"] = season
    if group:
        params["group"] = group
    if stats=="gameLog" and gamePk:
        params["gamePk"] = gamePk
        
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    if not data["stats"] or not data["stats"][0]["splits"]:
        return None

    splits = data["stats"][0]["splits"]

    if stats == "gameLog" and gamePk is not None:
        for split in splits:
            if split["game"]["gamePk"] == gamePk:
                return split
        return None  # GamePk not found

    # For 'season' or all game logs
    return splits[0] if splits else None


In [5]:
#Normalize name for pulling special fields from boxscore
def normalize_name(name_raw):
    if ',' in name_raw:
        last, first = [part.strip() for part in name_raw.split(',', 1)]
        return f"{first} {last}".strip()
    return name_raw

In [6]:
# Get schedule for start/end date (regular season)

games = []

schedule = statsapi.schedule(
    str(start_date),
    str(end_date),
    sportId=1
)
games = []
weather = []
print(len(schedule))

#Pull available game data from schedule
for game in schedule:
    gamePk = game['game_id']
    
    
    #Get weather
    boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
    for item in boxscore['info']:
        if item['label'] == "Weather":
            weather = item['value']
        elif item['label'] == "Att":
            attendance = item['value']
        
    games.append({
        "gamePk": game['game_id'],
        "gametime": game['game_datetime'],
        "season": int(game['game_date'][:4]),
        "awayId": game['away_id'],
        "awayName": game['away_name'],
        "homeId": game['home_id'],
        "homeName": game['home_name'],
        "weather": weather,
        "attendance": attendance or None,
        "venue": game['venue_name']
    })   
    

15


In [7]:
teams = statsapi.get("teams", params = {"sportId":1})
raw_teams = teams['teams']
teams = []
for t in raw_teams:
    teams.append({
        'teamId': t['id'],
        'name': t['clubName'],
        'league': t['league']['name'],
        'venue': t['venue']['name'],
        'division': t['division']['name']
    })        
    

In [8]:
#Create hash PK for GAME_STATS table
def hash_game_stat(entity, stat, gamePk=None):
    key = f"{entity}:{stat}:{gamePk}" if gamePk is not None else f"{entity}:{stat}"
    return hashlib.sha256(key.encode('utf-8')).hexdigest()

game_stats = []

seen_players = set()
#Get team and player stats for games in games through boxscore endpoint
for g in games:
    gamePk = g['gamePk']
    gameDate = g['gametime'][:10]
    boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
    for side in ["home", "away"]:
        team_stats = boxscore["teams"][side]["teamStats"]
        team_id = boxscore["teams"][side]['team']['id']
        # sys.exit()
        #Add team stats for games in schedule from boxscore
        for group in ['batting','pitching','fielding']:
            group_stats = team_stats.get(group, {})
            for stat, val in group_stats.items():
                game_stats.append({
                    "statsKey": hash_game_stat(team_id,stat,gamePk), 
                    "entity": team_id,
                    "scope": 'team',
                    "group": group,
                    "stat": stat, 
                    "val": val, 
                    "gamePk": gamePk,
                    "positionsPlayed": None
                })
            #print(f"Loaded {side} team data from game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
            
            #Player stats from boxscore 
            
            for player_key, player_data in boxscore["teams"][side]["players"].items():
                player_id = player_data["person"]["id"]  # ✅ This is the integer ID
                player_name = player_data["person"]["fullName"] #Thanks chat 
                position = player_data["position"]["abbreviation"]
                
                for group in ["batting", "fielding", "pitching"]:
                    group_stats = player_data["stats"].get(group, {})
                    if len(group_stats) == 0: #If player did not hit or pitch or field, no stats
                        continue
                    for stat, val in group_stats.items():
                        game_stats.append({
                            "statsKey": hash_game_stat(player_id,stat,gamePk),
                            "entity":player_id,
                            "scope": 'player',
                            "group":group,
                            "stat":stat, 
                            "val":val, 
                            "gamePk":gamePk,
                            "positionsPlayed": position
                        })
                #print(f"Loaded player data for {player_name} {player_id} in game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
                seen_players.add(player_name)
len_players = len(seen_players)

In [9]:
#Create hash PK for season_stats table
def hash_szn_stat(entity, stat, group, season=None):
    key = f"{entity}:{stat}:{gamePk}:{group}" if gamePk is not None else f"{entity}:{stat}:{group}"
    
    return hashlib.sha256(key.encode('utf-8')).hexdigest()


In [ ]:


#Get player season stats
season_stats = []

len_players = len(seen_players)
seen_players = set()
for stats in game_stats:
    if stats['scope'] != 'player' or stats['entity'] in seen_players:
        continue
    else:
        playerId = stats['entity']
    i_players = len(seen_players)

    
    person = statsapi.get("person", params={"personId": playerId})
    
    for group in ["hitting", "pitching", "fielding"]:
        data = player_stat_data(playerId, currentSeason)
        
        for stat, val in data['stat'].items():
            stat_key = hash_szn_stat(playerId, stat, group)
            season_stats.append({
                "statsKey": stat_key,
                "entity": playerId,
                "scope": "player",
                "group": group,
                "stat":stat, 
                "val":val,  
                "season":currentSeason,
                "updateDate":today,
            })
    
    seen_players.add(playerId)
    print(f"Got {currentSeason} season stats for {playerId} ({i_players} out of {len_players})")
            

    
    

In [ ]:

#Team season stats


def team_stat_data(team_id, group, season):
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats"
    params = {
        "season": season,
        "group": group,
        "stats": "season",
        "updateDate": today
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()["stats"][0]["splits"][0]


hash_szn_stats = []
for team in teams:
    teamId = team['teamId']
    teamName = team['name']
    for group in ["hitting","pitching","fielding"]:
    
        team_season_stats = team_stat_data(teamId, group, currentSeason)
        if 'stat' in team_stat_data(teamId, group, currentSeason):
            for stat, val in team_season_stats['stat'].items():         
                stat_key = hash_szn_stat(teamId,stat,currentSeason)
                season_stats.append({
                    "statsKey": stat_key,
                    "entity": teamId,
                    "scope": "team",
                    "group": group,
                    "stat": stat,
                    "val": val,
                    "season":currentSeason,
                    "updateDate": today
                })
            print(f"Stored team {group} season stats for {teamName}")
                
            
            


In [22]:
def is_valid_pitch(event):
    event_type = event.get("type")
    is_pitch_type = event_type == 'pitch'
    return is_pitch_type

def pbp_hash_key(gamePk, index, atBatIndex=None):
    key = f"{gamePk}:{index}:{atBatIndex}" if atBatIndex is not None else f"{gamePk}:{index}"
    return hashlib.sha256(key.encode('utf-8')).hexdigest()


at_bats = []
all_sequences = []
#Games (list of str)
for game in games[0]:
    pbp = statsapi.get("game_playByPlay", params={"gamePk": gamePk})
    all_plays = pbp['allPlays']
    game_sequence = []
    for atbat in all_plays:
        sequence = []
        pprint(atbat)
        sys.exit()
        index = atbat['atBatIndex']
        batter=atbat["matchup"]["batter"]["id"]
        pitcher= atbat["matchup"]["pitcher"]["id"]

        at_bats.append({
            "atbat_key": pbp_hash_key(gamePk, index),
            "game_index": index,
            'batterId': batter,
            'pitcherId': pitcher,
            'splits': atbat['splits'],
            'inning': atbat['inning']
                })
        # season_stats = player_stat_data(pitcher, group = "pitching", season = 2025)
        
        for event in atbat.get("playEvents", []):
            
            if not is_valid_pitch(event):
                continue
            else:
                # Extract relevant pitch info
                pitch_number = event.get("pitchNumber")
                count = event.get("count", {})
                result = event.get("details", {}).get("description", "")
                pitch_type = event.get("details", {}).get("type", {}).get("description", "")
                speed = event.get("pitchData", {}).get("startSpeed", None)
                # Append to context
                sequence.append({
                    "pitchNumber": pitch_number,
                    "balls": count.get("balls", 0),
                    "strikes": count.get("strikes", 0),
                    "outs": count.get("outs", 0),
                    "result": result,
                    "pitchType": pitch_type,
                    "speed": speed,
                    "batter": batter,
                    "batter_side": atbat['matchup']['batSide']['code'],
                    "pitcher": pitcher,
                    "pitcher_hand": atbat['matchup']['pitchHand']['code'],
                    "result": atbat["result"]["eventType"],
                    "pitch_sequence": sequence,
                    # "pitches_per_inning": season_stats['stat']['pitchesPerInning'],
                    "type_confidnce":event.get("pitchData",{}).get('typeConfidence',{})
                })
        game_sequence.append(sequence)
    all_sequences.append(game_sequence)


777103
{'about': {'atBatIndex': 0,
           'captivatingIndex': 0,
           'endTime': '2025-07-19T02:10:43.699Z',
           'halfInning': 'top',
           'hasOut': True,
           'hasReview': False,
           'inning': 1,
           'isComplete': True,
           'isScoringPlay': False,
           'isTopInning': True,
           'startTime': '2025-07-19T02:10:18.272Z'},
 'actionIndex': [0, 1, 2],
 'atBatIndex': 0,
 'count': {'balls': 1, 'outs': 1, 'strikes': 0},
 'matchup': {'batSide': {'code': 'L', 'description': 'Left'},
             'batter': {'fullName': 'Brice Turang',
                        'id': 668930,
                        'link': '/api/v1/people/668930'},
             'batterHotColdZones': [],
             'pitchHand': {'code': 'R', 'description': 'Right'},
             'pitcher': {'fullName': 'Tyler Glasnow',
                         'id': 607192,
                         'link': '/api/v1/people/607192'},
             'pitcherHotColdZones': [],
             'sp

SystemExit: 

In [15]:
with sqlite3.connect('database/mlb_db.db', timeout = 10) as conn:
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS GAMES;")
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS GAMES (
    gamePk INTEGER PRIMARY KEY,
    gametime TEXT,
    season INTEGER, 
    awayId INTEGER,
    homeId INTEGER,
    weather TEXT,
    attendance TEXT,
    venue TEXT
    );
    """)
    
    cursor.executemany("""
    INSERT OR IGNORE INTO GAMES (gamePk, gametime, season, awayId, homeId, weather, attendance, venue)
        VALUES (:gamePk, :gametime, :season, :awayId, :homeId, :weather, :attendance, :venue)
        ;
    """, games)
    cursor.execute("DROP TABLE IF EXISTS TEAMS;")
    cursor.execute("""            
    CREATE TABLE IF NOT EXISTS TEAMS (
    teamId INTEGER PRIMARY KEY,
    teamName TEXT,
    league TEXT,
    venue TEXT,
    division TEXT
    );
    """)
    
    conn.commit()
    cursor.executemany("""
    INSERT OR IGNORE INTO TEAMS (teamId, teamName, league, venue, division) 
    VALUES (:teamId, :name, :league, :venue, :division)
    ;
    """, teams)
    
    conn.commit() 
    cursor.execute("DROP TABLE IF EXISTS GAME_STATS;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS GAME_STATS (
    statsKey TEXT PRIMARY KEY,
    entity TEXT,
    scope BINARY, 
    "group" TEXT, --GROUP is SQL keyword
    stat TEXT,
    val TEXT,
    gamePk INTEGER, 
    positionsPlayed TEXT
    );
    """)
    
    cursor.executemany("""
    INSERT OR IGNORE INTO GAME_STATS (statsKey, entity, scope, "group", stat, val, gamePk, positionsPlayed)
    VALUES(:statsKey, :entity, :scope, :group, :stat, :val, :gamePk, :positionsPlayed)
    ;
    """, game_stats)
    conn.commit()
    cursor.execute("DROP TABLE IF EXISTS SEASON_STATS;")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS SEASON_STATS (
    statsKey TEXT PRIMARY KEY,
    entity TEXT,
    scope TEXT,
    "group" TEXT,
    stat TEXT,
    val TEXT,
    season INTEGER,
    updateDate DATETIME
    );
    """)
    conn.commit()

    cursor.executemany("""
    INSERT OR IGNORE INTO SEASON_STATS (statsKey, entity, scope, "group", stat, val, season, updateDate)
    VALUES(:statsKey, :entity, :scope, :group, :stat, :val, :season, :updateDate)
    ;
    """, season_stats)
    conn.commit()
conn.close()

In [ ]:

examples = []
sequence = []
def build_pitch_context(atbat_data,season_stats):
    for pitch in atbat_data:
        current_pitch = pitch.get("pitchType")
    
        # Only create example if there's at least one prior pitch in the game
        if sequence:
            examples.append({
                "X": sequence.copy(),  # Copy to prevent mutation
                "y": current_pitch,
                "context": {
                    "balls": pitch["balls"],
                    "strikes": pitch["strikes"],
                    "outs": pitch["outs"],
                    "batter_side": pitch["batter_side"],
                    "pitcher_hand": pitch["pitcher_hand"],
                    "pitches_per_inning": pitch["pitches_per_inning"].c
                }
            })
    
        sequence.append(current_pitch) #First pitch will have no context

